In [2]:
%matplotlib inline
import numpy as np
import theano
import theano.tensor as T
theano.config.exception_verbosity='high'
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import cPickle
import gzip
import os
import sys
import timeit

In [261]:
class HiddenLayer(object):
    """
    Implementation of hidden layer class. Source: http://deeplearning.net/tutorial/mlp.html#mlp.

     :type rng: numpy.random.RandomState
    :param rng: a random number generator used to initialize weights

    :type input: theano.tensor.dmatrix
    :param input: a symbolic tensor of shape (n_examples, n_in)

    :type n_in: int
    :param n_in: dimensionality of input

    :type n_out: int
    :param n_out: number of hidden units

    :type activation: theano.Op or function
    :param activation: Non linearity to be applied in the hidden layer

    """

    def __init__(self, rng, input, n_in, n_out, W=None, b=None, activation=T.tanh):
        self.input = input
        
        if W is None:
            W_values = np.asarray(
                rng.uniform(
                    low = -np.sqrt(6. / (n_in + n_out)), 
                    high = np.sqrt(6. / (n_in + n_out)), 
                    size = (n_in, n_out)
                ),
                dtype = theano.config.floatX)
            
            if activation == theano.tensor.nnet.sigmoid:
                W_values *= 4
                
            W = theano.shared(value=W_values, name='W', borrow=True)
            
        if b is None:
            b_values = np.zeros((n_out,), dtype = theano.config.floatX)
            b = theano.shared(value = b_values, name='b', borrow=True)
        
        self.W = W
        self.b = b
        
        self.activation = activation
        
        lin_output = T.dot(input, self.W) + self.b
        self.output = (
            lin_output if activation is None
            else activation(lin_output)
        )
        
        self.params = [self.W, self.b]


In [7]:
'''
Simulates cartpole starting at x0 with action u -- ported from CS287 Matlab code
'''

def sim_cartpole(x0, u, dt):
    
    def dynamics(x, u):
        mc = 10
        mp = 1
        l = 0.5
        g = 9.81
        T = 0.25
        s = np.sin(x[1])
        c = np.cos(x[1])
        
        xddot = (u + np.multiply(mp*s, l*np.power(x[3],2) + g*c))/(mc + mp*np.power(s,2))
        tddot = (-u*c - np.multiply(np.multiply(mp*l*np.power(x[3],2), c),s) - 
                 np.multiply((mc+mp)*g,s)) / (l * (mc + np.multiply(mp, np.power(s,2))))
        xdot = x[2:4]
        xdot = np.append(xdot, xddot)
        xdot = np.append(xdot, tddot)
        
        return xdot
    
    DT = 0.1
    t = 0
    while t < dt:
        current_dt = min(DT, dt-t)
        x0 = x0 + current_dt * dynamics(x0, u)
        t = t + current_dt
    
    return x0
    
'''
Linearizes the dynamics of cartpole around a reference point for use in an LQR controler
'''

def linearize_cartpole(x_ref, u_ref, dt, eps):
    A = np.zeros([4,4])

    for i in range(4):
        increment = np.zeros([4,])
        increment[i] = eps
        A[:,i] = (sim_cartpole(x_ref + increment, u_ref, dt) - 
                  sim_cartpole(x_ref, u_ref, dt)) / (eps)
    
    B = (sim_cartpole(x_ref, u_ref + eps, dt) - sim_cartpole(x_ref, u_ref, dt)) / (eps)
    
    c = x_ref
    
    return A, B, c

'''
Computes the LQR infinte horizon controller associated with linear dyamics A, B and quadratic cost Q, R

NOTE: Current version only works for cartpole because I hardcoded a couple of numbers for now
'''

def lqr_infinite_horizon(A, B, Q, R):
    nA = A.shape[0]

    if len(B.shape) == 1:
        nB = 1
    else:
        nB = B.shape[1]

    P_current = np.zeros([nA, nA])

    P_new = np.eye(nA)

    K_current = np.zeros([nB, nA])

    K_new= np.triu(np.tril(np.ones([nB,nA]),0),0)

    while np.linalg.norm(K_new - K_current, 2) > 1E-4:
        P_current = P_new
      
        K_current = K_new
        
        Quu = R + np.dot(np.dot( np.transpose(B), P_current), B)
        
        K_new = -np.linalg.inv(Quu) * np.dot(np.dot( np.transpose(B), P_current), A)
    
        P_new = Q + np.dot(np.dot( np.transpose(K_new), 
                                   R), 
                                   K_new) + np.dot(np.dot( np.transpose(A + np.dot(B.reshape(4,1), K_new)),
                                                           P_current),
                                                           (A + np.dot(B.reshape(4,1), K_new.reshape(1,4)))
                          )
        
    return K_new, P_new, Quu

'''
Generate LQR trajectory for solving our simple cartpole problem
'''
x_ref = np.array([0, np.pi, 0, 0])
u_ref = 0.
A, B, c = linearize_cartpole(x_ref, u_ref, 0.1, 0.1)
Q = np.eye(4)
R = np.eye(1)
dt = 0.1
x_init = np.array([0, np.pi - np.pi/10, 0, 0])

K_inf, P_inf, Quu = lqr_infinite_horizon(A, B, Q, R)

x_lqr = np.zeros([4,500])
u_lqr = np.zeros([1,500])

x_lqr[:,0] = np.array([0, np.pi - np.pi/10, 0, 0])
u_lqr[:,0] = np.dot(K_inf, (x_lqr[:,0] - x_ref)) + u_ref

for i in range(499):
    x_lqr[:,i+1] = sim_cartpole(x_lqr[:,i], u_lqr[:,i], dt)
    u_lqr[:,i+1] = np.dot(K_inf, (x_lqr[:,i] - x_ref) ) + u_ref

    
'''
Function to generate samples from the guidance trajectory
'''
def gen_traj_guidance(x_init, x_ref, u_ref, K, variance, traj_size, dt):
    xs = len(x_ref)
    
    if type(u_ref) == float:
        us = 1
    else:
        us = len(u_ref)
    
    x_traj = np.zeros([xs, traj_size])
    u_traj = np.zeros([us, traj_size])
    
    x_traj[:,0] = x_init
    u_traj[:,0] = np.random.multivariate_normal(np.dot(K, (x_traj[:,0] - x_ref) ) + u_ref, variance)
    
    for t in range(traj_size-1):
        x_traj[:,t+1] = sim_cartpole(x_traj[:,t], u_traj[:,t], dt)
        u_mean = np.dot(K, (x_traj[:,t] - x_ref) ) + u_ref
        u_traj[:,t+1] = np.random.multivariate_normal(u_mean, variance)
    
    return x_traj, u_traj

x_init_options = [
    np.array([0, np.pi - np.pi/4, 0, 0]),
    np.array([10, np.pi - np.pi/4, 0, 0]),
    np.array([-10, np.pi - np.pi/4, 0, 0]),
    np.array([0, np.pi + np.pi/4, 0, 0]),
    np.array([10, np.pi + np.pi/4, 0, 0]),
    np.array([-10, np.pi + np.pi/4, 0, 0]),
]



In [8]:
class SimpleNN(object):
    ''' Simple Neural Net class
    
    Single hidden layer net with one layer of hidden units and nonlinear activations. Top layer is linear
    Source: http://deeplearning.net/tutorial/mlp.html#mlp
    '''
    
    def __init__(self, rng, input, n_in, n_hidden, n_out):
        """Initialize the parameters for the multilayer perceptron

        :type rng: numpy.random.RandomState
        :param rng: a random number generator used to initialize weights

        :type input: theano.tensor.TensorType
        :param input: symbolic variable that describes the input (one minibatch)

        :type n_in: int
        :param n_in: number of input units
        
        :type n_hidden: int
        :param n_hidden: number of hidden units

        :type n_out: int
        :param n_out: number of output units

        """
                
        self.hiddenLayer = HiddenLayer(
            rng = rng,
            input=input,
            n_in = n_in, 
            n_out = n_hidden, 
            activation = T.tanh
        )
        
        self.outputLayer = HiddenLayer(
            rng = rng,
            input = self.hiddenLayer.output,
            n_in = n_hidden,
            n_out = n_out,
            activation = None
        )
             
        
        ## L1 and L2 regularization
        self.L1 = ( abs(self.hiddenLayer.W).sum() + abs(self.outputLayer.W).sum() )
        
        self.L2_sqr = ( (self.outputLayer.W ** 2).sum() + (self.hiddenLayer.W ** 2).sum())
        
        # neg log likelihood is given by that of the output
        #self.negative_log_likelihood = ( self.logRegressionLayer.negative_log_likelihood )
        #self.errors = self.logRegressionLayer.errors
        
        self.params = self.hiddenLayer.params + self.outputLayer.params
        
        self.input = input
        
        self.output = self.outputLayer.output
        
    def meanSqErr(self, u):
        if u.ndim != self.output.ndim:
            raise TypeError(
                'u should have the same shape as self.output',
                ('u', u.type, 'self.output', self.output.type)
            )
        return T.mean( (self.output - u)**2 )

In [185]:
def train_NN(learning_rate=0.01, L1_reg=0.0, L2_reg=0.0001, n_epochs=1000, batch_size=20, n_hidden=20, 
             LQR_controller=K_inf, LQR_start=x_init, LQR_var=Quu, num_traj=10):
    
    #########################
    # GENERATE TRAJECTORIES #
    #########################
    
    x_traj_list = []
    u_traj_list = []
    # Generate num_traj sample trajectories from our LQR policy for each of training, validation, and test
    if type(LQR_start) == list:
        n_guidance = len(LQR_start)
        for i in range(3): # training, test, validiation
            for j in range(n_guidance): # starting positions
                for k in range(num_traj): # generate this many trajectories
                    x_traj1, u_traj1 = gen_traj_guidance(LQR_start[j], x_ref, u_ref, LQR_controller, LQR_var, 500, dt)
                    x_traj_list.append(x_traj1)
                    u_traj_list.append(u_traj1)
    else:
        n_guidance = 1
        for t in range(3*num_traj):
            x_traj1, u_traj1 = gen_traj_guidance(LQR_start, x_ref, u_ref, LQR_controller, LQR_var, 500, dt)
            x_traj_list.append(x_traj1)
            u_traj_list.append(u_traj1)
        
    train_set_x = theano.shared(
        value = np.concatenate(x_traj_list[:n_guidance*num_traj], axis=1).T, 
        name='tr_x', borrow=True)
    train_set_u = theano.shared(
        value = np.concatenate(u_traj_list[:n_guidance*num_traj], axis=1).T, 
        name='tr_u', borrow=True)
    valid_set_x = theano.shared(
        np.concatenate(x_traj_list[n_guidance*num_traj:2*n_guidance*num_traj], axis=1).T, 
        name='v_x', borrow=True)
    valid_set_u = theano.shared(
        np.concatenate(u_traj_list[n_guidance*num_traj:2*n_guidance*num_traj], axis=1).T, 
        name='v_u', borrow=True)
    test_set_x = theano.shared(
        np.concatenate(x_traj_list[2*n_guidance*num_traj:3*n_guidance*num_traj], axis=1).T, 
        name='te_x', borrow=True)
    test_set_u = theano.shared(
        np.concatenate(u_traj_list[n_guidance*num_traj:2*n_guidance*num_traj], axis=1).T, 
        name='te_u', borrow=True)
    
    #############################################
    # SIMPLER VERSION: TRYING TO LEARN DET. LQR #
    #############################################
    #train_set_x = theano.shared(value = x_lqr.T, name='tr_x', borrow=True)
    #train_set_u = theano.shared(value = u_lqr.T, name='tr_u', borrow=True)
    #valid_set_x = theano.shared(value = x_lqr.T, name='v_x', borrow=True)
    #valid_set_u = theano.shared(value = u_lqr.T, name='v_u', borrow=True)
    #test_set_x = theano.shared(value = x_lqr.T, name='te_x', borrow=True)
    #test_set_u = theano.shared(value = u_lqr.T, name='t_u', borrow=True)


    # compute number of minibatches for training, validation and testing
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
    n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size

    ######################
    # BUILD ACTUAL MODEL #
    ######################
    print '... building the model'

    # allocate symbolic variables for the data
    index = T.lscalar()  # index to a [mini]batch
    x = T.matrix('x')  # the data is presented as a matrix of positions
    u = T.matrix('u')  # control inputs are a 1d matrix

    rng = np.random.RandomState(1234)

    # construct the MLP class
    classifier = SimpleNN(
        rng=rng,
        input=x,
        n_in=4,
        n_hidden=n_hidden,
        n_out=1
    )

    # define cost function
    cost = (
        classifier.meanSqErr(u)
        + L1_reg * classifier.L1
        + L2_reg * classifier.L2_sqr
    )
    
    # compile a Theano function that computes the mistakes that are made by the model on a minibatch
    test_model = theano.function(
        inputs=[index],
        outputs=cost,
        givens={
            x: test_set_x[index * batch_size:(index + 1) * batch_size],
            u: test_set_u[index * batch_size:(index + 1) * batch_size]
        }
    )

    validate_model = theano.function(
        inputs=[index],
        outputs=cost,
        givens={
            x: valid_set_x[index * batch_size:(index + 1) * batch_size],
            u: valid_set_u[index * batch_size:(index + 1) * batch_size]
        }
    )
    
    # calculate gradient
    gparams = [T.grad(cost, param) for param in classifier.params]
    
    # rule for parameter updates
    updates = [
        (param, param- learning_rate * gparam)
        for param, gparam in zip(classifier.params, gparams)
    ]
    
    # return cost and update parameters
    train_model = theano.function(
        inputs=[index],
        outputs=cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
            u: train_set_u[index * batch_size: (index + 1) * batch_size]
        }
    )
    
    ###############
    # TRAIN MODEL #
    ###############
    print '... training'

    # early-stopping parameters
    patience = 10000000  # look as this many examples regardless (10000)
    patience_increase = 2  # wait this much longer when a new best is found
    improvement_threshold = 0.995  # a relative improvement of this much is considered significant
    validation_frequency = min(n_train_batches, patience / 2)

    best_validation_loss = np.inf
    best_iter = 0
    test_score = 0.
    start_time = timeit.default_timer()

    epoch = 0
    done_looping = False

    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in xrange(n_train_batches):

            minibatch_avg_cost = train_model(minibatch_index)
            # iteration number
            iter = (epoch - 1) * n_train_batches + minibatch_index

            if (iter + 1) % validation_frequency == 0:
                # compute zero-one loss on validation set
                validation_losses = [validate_model(i) for i
                                     in xrange(n_valid_batches)]
                this_validation_loss = np.mean(validation_losses)
                
                if (iter + 1) % (validation_frequency * 100.) == 0:
                    print(
                        'epoch %i, minibatch %i/%i, validation error %f' %
                        (
                            epoch,
                            minibatch_index + 1,
                            n_train_batches,
                            this_validation_loss
                        )
                    )

                # if we got the best validation score until now
                if this_validation_loss < best_validation_loss:
                    #improve patience if loss improvement is good enough
                    if (
                        this_validation_loss < best_validation_loss *
                        improvement_threshold
                    ):
                        patience = max(patience, iter * patience_increase)

                    best_validation_loss = this_validation_loss
                    best_iter = iter

                    # test it on the test set
                    test_losses = [test_model(i) for i
                                   in xrange(n_test_batches)]
                    test_score = np.mean(test_losses)
                    if (iter +1 % (validation_frequency * 100.) == 0):
                        print(('     epoch %i, minibatch %i/%i, test error of '
                               'best model %f') %
                              (epoch, minibatch_index + 1, n_train_batches,
                               test_score))

            if patience <= iter:
                done_looping = True
                break

    end_time = timeit.default_timer()
    print(('Optimization complete. Best validation score of %f '
           'obtained at iteration %i, with test performance %f') %
          (best_validation_loss, best_iter + 1, test_score))
    print 'The code ran for %d epochs, with %f epochs/sec' % (
        epoch, 1.*epoch / (end_time - start_time))
    
    return x, classifier
    
    

In [215]:
x, policy = train_NN(learning_rate=0.00005, L1_reg=0.0, L2_reg=0.0000, n_epochs=10000, batch_size=50, n_hidden=20, 
                     LQR_controller=K_inf, LQR_start=x_init_options, LQR_var=Quu, num_traj=10)

... building the model
... training
epoch 100, minibatch 600/600, validation error 187.451360
epoch 200, minibatch 600/600, validation error 110.169057
epoch 300, minibatch 600/600, validation error 67.629199
epoch 400, minibatch 600/600, validation error 35.693192
epoch 500, minibatch 600/600, validation error 47.166132
epoch 600, minibatch 600/600, validation error 15.588173
epoch 700, minibatch 600/600, validation error 15.600323
epoch 800, minibatch 600/600, validation error 14.363079
epoch 900, minibatch 600/600, validation error 12.071353
epoch 1000, minibatch 600/600, validation error 12.771165
epoch 1100, minibatch 600/600, validation error 10.166929
epoch 1200, minibatch 600/600, validation error 9.855234
epoch 1300, minibatch 600/600, validation error 9.238201
epoch 1400, minibatch 600/600, validation error 9.138170
epoch 1500, minibatch 600/600, validation error 8.741061
epoch 1600, minibatch 600/600, validation error 8.121983
epoch 1700, minibatch 600/600, validation error 

In [216]:
feed_forward = theano.function(
    inputs=[x],
    outputs=policy.output)

In [270]:
dt = 0.1
x_init2 = np.array([3, np.pi - np.pi/4, 0, 0])

x_sim = np.zeros([4,500])
x_sim[:,0] = x_init2

u_sim = np.zeros([500,])
u_sim[0] = feed_forward(x_sim[:,0].reshape([1,4]))

for t in range(499):
    x_sim[:,t+1] = sim_cartpole(x_sim[:,t], u_sim[t], dt)
    u_sim[t+1] = feed_forward(x_sim[:,t+1].reshape([1,4]))

In [271]:
x_sim[:,-1]

array([  2.00967144e-02,   3.14160114e+00,  -7.50992603e-04,
        -1.21293118e-05])